# Lab | API Wrappers




In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint  # for random wait time respectful scraping
from time import sleep      # sleep function

## Lab 6.01

In [ ]:
url = "https://www.billboard.com/charts/hot-100/"
response = requests.get(url)
response.status_code # 200 status code means OK!


In [ ]:
soup = BeautifulSoup(response.content, "html.parser")
# soup

In [ ]:
#post-1479786 > div.pmc-paywall > div > div > div > div.chart-results-list.\/\/.lrv-u-padding-t-150.lrv-u-padding-t-050\@mobile-max > div:nth-child(14) > ul > li.lrv-u-width-100p > ul > li.o-chart-results-list__item.\/\/.lrv-u-flex-grow-1.lrv-u-flex.lrv-u-flex-direction-column.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0\@mobile-max.lrv-u-border-color-grey-light.lrv-u-padding-l-050.lrv-u-padding-l-1\@mobile-max
# ul > li.o-chart-results-list__item


In [ ]:
#soup.select("#post-1479786 > div.pmc-paywall > div > div > div > div.chart-results-list.\/\/.lrv-u-padding-t-150.lrv-u-padding-t-050\@mobile-max > div:nth-child(8) > ul > li.lrv-u-width-100p > ul > li.o-chart-results-list__item.\/\/.lrv-u-flex-grow-1.lrv-u-flex.lrv-u-flex-direction-column.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0\@mobile-max.lrv-u-border-color-grey-light.lrv-u-padding-l-050.lrv-u-padding-l-1\@mobile-max")


In [ ]:
#soup.select("li.o-chart-results-list__item")[0]

In [ ]:
# for i in soup.select("li.o-chart-results-list__item > h3"):    # specific syntax
#     print(i.get_text())

In [ ]:
len(soup.select("li.o-chart-results-list__item > h3"))
len(soup.select("li.o-chart-results-list__item > span:nth-child(2)"))
len(soup.select("li.lrv-u-width-100p > ul > li.o-chart-results-list__item:nth-child(1)"))

In [ ]:
song_artist = []
for i in soup.select("li.lrv-u-width-100p > ul > li.o-chart-results-list__item:nth-child(1)"):
    #print(i.get_text())
    song_artist.append(i.get_text())

In [ ]:
#song_artist

In [ ]:
song = []
artist = []
for i in song_artist:
    i = i.split('\n\n\n')
    if len(i) == 2:
        artist.append(i[1].replace('\n\n','').lstrip())
        song.append(i[0].replace('\n\n',''))

In [ ]:
song[0:10]

In [ ]:
artist[0:10]

In [ ]:
hot100 = pd.DataFrame({"song":song,
                       "artist":artist
                      })

In [ ]:
# YEAH ,I cracked the Billboard Hot-100 !!!!!
hot100.head(5)

## Lab 6.02

In [ ]:
# # using header to avoind 403 error, server thinks I am a just a user browsing
# header = {
#   "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
#   "X-Requested-With": "XMLHttpRequest"
# }

# artist = []
# song = []
# years = range(1969, 2017)

# for y in years:
#     start_at= str(y)
#     wait_time = randint(1,4)
#     sleep(wait_time)
#     url = "https://playback.fm/charts/top-100-songs/" + start_at 
#     response = requests.get(url, headers=header)
#     soup = BeautifulSoup(response.content, "html.parser")
#     for i in soup.select("td:nth-child(2) > a"):
#         i = i.get_text()
#         i = i.replace('\n','')
#         artist.append(i)    
#     for i in soup.select("td.mobile-hide > a > span.song"):
#         i = i.get_text()
#         song.append(i) 
#     print("processed:", url, "with wait time:", wait_time, "- songs:", len(song), "artists:", len(artist))
# top_100 = pd.DataFrame({"song":song, "artist":artist})  
# top_100.shape

In [ ]:
# top_100.head()

## Lab | API Wrappers

<br>

<details><summary>▶ Instructions:</summary>
<p>

To move forward with the project, you need to create a collection of songs with their audio features - as large as possible!

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

</p>
</details>

In [1]:
import requests
import pandas as pd
from time import sleep 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
import time

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()

In [3]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]
# secrets_dict

In [4]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [5]:
def get_playlist_tracks(playlist_id):
    init_s = time.time()
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        init_e = time.time()
        if init_e-init_s > 100:   # re-init at 100 sec.
            spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))       
            print(init_s,init_e)
            init_s = time.time()
        sleep(randint(1,3))
    return tracks

In [6]:
# get some coffee
start = time.time()

lst_pl = ['4VwQ8oKy4iVbUa6TSDQEB0','0V8jbEIKC4ppJBHiCRdtRL','59KM8m44lmfnGW6khGAsn9',   # test1, test5, classic
          '35i05dxUnfnU0ulnimZh3V','37i9dQZF1DWTmvXBN4DgpA',#'29isGIzk5nnq9J9fnjDgD4',   # jazz, top-2000, longest 
          '1rpa9cJm4ZPW2hhcuHfhvv','03sDEv7FN58Mb9CJOs1Tgn','0mStf5UM1VDU0ShoxNf11m',   # metal, reggeaton
          '5SqR3iQ1rvzjjB8vEPlF8d','4hdH7uPmzHCt0FGPBKOV4G','37i9dQZF1DX4AyFl3yqHeK'    # bachata, merengue, bossa nova
         ]

title = []            # track name
artst = []            # artist name
t_uri = []            # song uri
popul = []            # popularity
dance = []            # danceability
energ = []            # energy
loudn = []            # loudness
tmode = []            # mode
speec = []            # speechiness
acous = []            # acousticness
instr = []            # instrumentalness
liven = []            # liveness
valen = []            # valence
tempo = []            # tempo
ttype = []            # type
durat = []            # duration_ms
times = []            # time_signature
tr_id = []            # song/track id

for p in lst_pl:
    n = 0                 # just a counter
    all_tracks = get_playlist_tracks(p)
    print('Songs received from spotify ('+p+')') 
    for track in all_tracks:
        start = time.time()
        # audio features
        audio_features = sp.audio_features(track["track"]["uri"])[0]  # only one time I/O
        title.append(track['track']['name'])
        tr_id.append(track["track"]["id"])
        t_uri.append(track["track"]["uri"])
        artst.append(track['track']['album']['artists'][0]['name'])
        popul.append(track['track']['popularity'])
        dance.append(audio_features['danceability'])
        energ.append(audio_features['energy'])
        loudn.append(audio_features['loudness'])
        tmode.append(audio_features['mode'])
        acous.append(audio_features['acousticness'])
        instr.append(audio_features['instrumentalness'])
        liven.append(audio_features['liveness'])
        valen.append(audio_features['valence'])
        tempo.append(audio_features['tempo'])
        durat.append(audio_features['duration_ms'])
        n += 1
        
    print('Features of',n,'songs assigned')
    
song_lake = pd.DataFrame({"song":title,
                          "name":artst,
                          "trackid":tr_id,
                          "trackuri":t_uri,
                          "popul":popul,
                          "dance":dance,
                          "energ":energ,
                          "loudn":loudn,
                          "tmode":tmode,
                          "acous":acous,
                          "instr":instr,
                          "liven":liven,
                          "valen":valen,
                          "tempo":tempo,
                          "durat":durat
                         })

print(time.time()-start)

Songs received from spotify (4VwQ8oKy4iVbUa6TSDQEB0)
Features of 1 songs assigned
Songs received from spotify (0V8jbEIKC4ppJBHiCRdtRL)
Features of 5 songs assigned
Songs received from spotify (59KM8m44lmfnGW6khGAsn9)
Features of 122 songs assigned
Songs received from spotify (35i05dxUnfnU0ulnimZh3V)
Features of 68 songs assigned
Songs received from spotify (37i9dQZF1DWTmvXBN4DgpA)
Features of 1997 songs assigned
Songs received from spotify (1rpa9cJm4ZPW2hhcuHfhvv)
Features of 78 songs assigned
Songs received from spotify (03sDEv7FN58Mb9CJOs1Tgn)
Features of 145 songs assigned
Songs received from spotify (0mStf5UM1VDU0ShoxNf11m)
Features of 142 songs assigned
Songs received from spotify (5SqR3iQ1rvzjjB8vEPlF8d)
Features of 207 songs assigned
Songs received from spotify (4hdH7uPmzHCt0FGPBKOV4G)
Features of 75 songs assigned
Songs received from spotify (37i9dQZF1DX4AyFl3yqHeK)
Features of 57 songs assigned
0.054431915283203125


In [7]:
song_lake.head(60)

,song,name,trackid,trackuri,popul,dance,energ,loudn,tmode,acous,instr,liven,valen,tempo,durat
0,"Bagatelle in A Minor, WoO 59 ""Für Elise""",Various Artists,50tFFbm0a9fp5K7tPTl0Gi,spotify:track:50tFFbm0a9fp5K7tPTl0Gi,38,0.3520,0.09490,-25.077,0,0.99300,0.912000,0.1060,0.0519,131.865,196013
1,Never Tear Us Apart,INXS,1GjbTNFImFrjFsNdleDe78,spotify:track:1GjbTNFImFrjFsNdleDe78,69,0.6640,0.61300,-7.560,1,0.00309,0.000128,0.1750,0.1930,96.600,184587
2,Need You Tonight,INXS,3h04eZTnmFLRMjZajbrp2R,spotify:track:3h04eZTnmFLRMjZajbrp2R,71,0.7950,0.63000,-7.217,0,0.04190,0.575000,0.0894,0.7850,108.701,181107
3,New Sensation,INXS,2xcrseImDFEf8Urommws03,spotify:track:2xcrseImDFEf8Urommws03,66,0.6820,0.95400,-4.951,1,0.00712,0.000040,0.0520,0.9280,115.475,220160
4,Never Tear Us Apart,INXS,3pyTksNccLM1jRvzQ4zTke,spotify:track:3pyTksNccLM1jRvzQ4zTke,57,0.6640,0.61300,-7.560,1,0.00309,0.000128,0.1750,0.1930,96.600,184587
5,Original Sin,INXS,7G67ZJRQT9nn2Fa9vA6B32,spotify:track:7G67ZJRQT9nn2Fa9vA6B32,57,0.6850,0.63300,-7.259,1,0.01360,0.000106,0.0935,0.7310,131.397,317240
6,Theme From Schindler's List,John Williams,3DzMT13HJV7EsbVbNZZJZQ,spotify:track:3DzMT13HJV7EsbVbNZZJZQ,0,0.1170,0.13400,-24.162,0,0.86400,0.838000,0.0977,0.0769,83.830,255160
7,"Bizet: Carmen, WD 31, Act 1 Scene 4: No. 5, Ha...",Georges Bizet,2r1FiNXh5mDNEP8K07YRVp,spotify:track:2r1FiNXh5mDNEP8K07YRVp,49,0.3630,0.11500,-17.325,1,0.97600,0.000000,0.0992,0.2310,134.424,262147
8,"Puccini: Madama Butterfly, Act 2: ""Un bel di v...",Giacomo Puccini,69EKwvg7nkdOO3RmeUMJZG,spotify:track:69EKwvg7nkdOO3RmeUMJZG,47,0.2070,0.17000,-11.784,1,0.99100,0.024300,0.0804,0.0827,77.235,287907
9,"Puccini: Gianni Schicchi, Act 1: ""O mio babbin...",Giacomo Puccini,78k57V2uo6Sm72j1H7Ih14,spotify:track:78k57V2uo6Sm72j1H7Ih14,49,0.1700,0.10800,-16.626,1,0.97200,0.071500,0.1530,0.0792,74.536,154507


In [8]:
len(song_lake)

2897